# [Session 1] Toolkit

The following repository contains various examples of how to carry out
efficient calculations in _C++_ I have collected over my PhD time.
General data processing may be executed in _Python_/_R_ whereas the core
of the program may be exported into the optimised low-level language for fast operations.


## Data handling

< > trivial

### Manual IO through streams

Let us take a look at the code of `example1.cpp`:

```cpp
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <filesystem>
```

In the main:

```cpp
// construct CWD-independent path to the input file
std::filesystem::path CWD = std::filesystem::current_path();
std::filesystem::path EXECPATH = (CWD / argv[0]).parent_path();
std::ifstream file(EXECPATH / "data/vector.txt");
```

```cpp
std::vector<float> data;
std::string line;
float temp;

while (std::getline(file, line)){
    std::istringstream ss(line);
    if (ss >> temp) data.push_back(temp);
}
file.close();

for (const float& value : data){
    std::cout << value << std::endl;
}
```

```
g++ example1.cpp -o example1 --std=c++17  && ./example1
```

Reading a matrix is quite similar: see `example2.cpp`:

```cpp
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <filesystem>

int main(int argc, char* argv[]){

    std::vector<std::vector<float>> data;
    std::string line;
    float temp;

    // construct CWD-independent path to the input file
    std::filesystem::path CWD = std::filesystem::current_path();
    std::filesystem::path EXECPATH = (CWD / argv[0]).parent_path();
    std::ifstream file(EXECPATH / "data/matrix.csv");

    while (std::getline(file, line)){
        std::istringstream ss(line);
        std::vector<float> row;
        while (ss >> temp){
            row.push_back(temp);
            if (ss.peek() == ',') ss.ignore();
        }
        data.push_back(row);
    }
    file.close();

    for (const std::vector<float>& row : data){
        for (int i=0; i<row.size(); ++i){
            std::cout << row[i];
            if (i!=row.size()-1) std::cout << ",";
        }
        std::cout << std::endl;
    }

    return 0;
}

```

```
g++ example2.cpp -o example2 --std=c++17  && ./example2
```

### readcsv library

https://github.com/d99kris/rapidcsv#more-examples

```cpp
#include <iostream>
#include <vector>
#include "rapidcsv.h"

int main(int argc, char* argv[]){

    // construct CWD-independent path to the input file
    std::filesystem::path CWD = std::filesystem::current_path();
    std::filesystem::path EXECPATH = (CWD / argv[0]).parent_path();
    rapidcsv::Document doc(
        EXECPATH / "data/matrix.csv",
        rapidcsv::LabelParams(-1, -1),
        rapidcsv::SeparatorParams(',')
    );

    int numRows = doc.GetRowCount();
    int numColumns = doc.GetColumnCount();

    float** data = new float*[numRows];
    for (int row = 0; row < numRows; ++row) data[row] = new float[numColumns];

    for (int row=0; row<numRows; ++row)
        for (int col=0; col<numColumns; ++col)
            data[row][col] = doc.GetCell<float>(col, row);

    for (int row = 0; row < numRows; ++row) delete[] data[row];
    delete[] data;

    for (int i=0; i<numColumns; ++i){
        std::vector<float> col = doc.GetColumn<float>(i);
        for (const float& value : col) std::cout << value << std::endl;
        std::cout << "===" << std::endl;

    }

    return 0;
}

```

```
g++ example3.cpp -o example3 --std=c++17  && ./example3
```

## Unit testing

Catch2: https://github.com/catchorg/Catch2

https://github.com/catchorg/Catch2/blob/devel/docs/benchmarks.md

https://github.com/catchorg/Catch2/blob/devel/docs/command-line.md#specify-the-number-of-benchmark-samples-to-collect

https://github.com/catchorg/Catch2/blob/devel/docs/Readme.md

example4src.cpp

```cpp
#include <cmath>
#include <cstdlib>
#include <ctime>

float F(float x){
    return (2*x + pow(2,x)) / x;
}

int G(int seed=time(nullptr)){
    srand(seed);
    return 1 + std::rand() % 100;
}

```

example4test.cpp

```cpp
#include <limits>
#include "example4src.cpp"
#include "catch_amalgamated.hpp"

TEST_CASE(
    "F: Execution",
    "[F]"
){
    REQUIRE_NOTHROW ( F(1) );
    REQUIRE ( std::isinf(F(0)) );

}

TEST_CASE(
    "F: Result",
    "[F]"
){
    REQUIRE ( F(2) == 4.0 );
    Catch::Approx target = Catch::Approx(12.0).epsilon(0.1);
    REQUIRE ( F(5.86) == target );
}

TEST_CASE(
    "F: Simple Benchmarking",
    "[F]"
){
    BENCHMARK( "0.1" ) { return F(0.1); };
    BENCHMARK( "10" ) { return F(10); };
    BENCHMARK( "-7" ) { return F(-7); };
}

TEST_CASE(
    "G: Fix-RNG",
    "[G]"
){
    REQUIRE ( G(0) == 31 );
}

```

```
g++ example4test.cpp catch_amalgamated.cpp  -o example4 --std=c++17

./example4 --durations yes --benchmark-samples 100 --benchmark-resamples 100000
```

## Real Analysis

### Boost

https://github.com/pulver/autodiff

https://www.boost.org/doc/libs/1_65_0/libs/math/doc/html/math_toolkit/roots/brent_minima.html

```cpp
#include <iostream>
#include <boost/math/differentiation/autodiff.hpp>
#include <boost/math/tools/minima.hpp>

template <typename T>
T x4(T const& x){
  T x4 = x * x;
  x4 *= x4;
  return x4;
}

struct F{
  double operator()(double const& x){
    return (x + 3) * (x - 1) * (x - 1);
  }
};

int main(){

  // automatic differentiation
  constexpr unsigned ord = 5;
  double arg = 0.1;
  auto const x = boost::math::differentiation::make_fvar<double, ord>(arg);
  auto const y = x4(x);
  for (unsigned i=0; i<=ord; ++i)
    std::cout <<
      "d(" << i << ")/dx [x^4] | x=" << arg << " | = " << y.derivative(i)
      << std::endl;

  std::cout << "=====" << std::endl;

  // mimimum search
  int bits = std::numeric_limits<double>::digits;
  double lower = -4.0;
  double upper = 4.0 / 3;
  std::pair<double, double> r = boost::math::tools::brent_find_minima(
    F(), lower, upper, bits
  );
  std::cout.precision(std::numeric_limits<double>::digits10);
  std::cout << "   min[F(x)] = " << r.second << std::endl;
  std::cout << "minarg[F(x)] = " << r.first << std::endl;

  return 0;
}

```

g++ example5.cpp -o example5 --std=c++17  && ./example5

### GSL

http://gnu.ist.utl.pt/software/gsl/manual/html_node/Numerical-Differentiation-Examples.html

https://www.gnu.org/software/gsl/doc/html/roots.html

https://www.gnu.org/software/gsl/doc/html/diff.html

```cpp
#include <iostream>
#include <gsl/gsl_errno.h>
#include <gsl/gsl_math.h>
#include <gsl/gsl_roots.h>
#include <gsl/gsl_deriv.h>

// f(x) = x^2 - 4
double function(double x, void* params){
    return x * x - 4.0;
}

int main(){

    // root search

    gsl_function F;
    F.function = &function;
    F.params = nullptr;
    double xLower = 1.0;
    double xUpper = 3.0;

    const gsl_root_fsolver_type* solverType = gsl_root_fsolver_brent;
    gsl_root_fsolver* solver = gsl_root_fsolver_alloc(solverType);
    gsl_root_fsolver_set(solver, &F, xLower, xUpper);

    int status;
    int iter = 0;
    int maxIter = 100;
    double root;
    double epsabs = 0.0;
    double epsrel = 1e-6;

    do{
        iter++;
        status = gsl_root_fsolver_iterate(solver);
        root = gsl_root_fsolver_root(solver);
        xLower = gsl_root_fsolver_x_lower(solver);
        xUpper = gsl_root_fsolver_x_upper(solver);
        status = gsl_root_test_interval(xLower, xUpper, epsabs, epsrel);
        if (status == GSL_SUCCESS){
            std::cout << "Function root found at x = " << root << std::endl;
            break;
        }
    } while (status == GSL_CONTINUE && iter < maxIter);

    gsl_root_fsolver_free(solver);

    std::cout << "=====" << std::endl;

    // derrivative
    double result, abserr;
    double arg = 2.1;
    double h = 1e-8;
    gsl_deriv_central(&F, arg, h, &result, &abserr);
    std::cout << "f'(x) = " << result << " +/- " << abserr << std::endl;

    return 0;
}

```

g++ example6.cpp -o example6 --std=c++17  -lgsl -lgslcblas && ./example6

## Linear Algebra

### Armadillo

https://github.com/adevress/armadillo/blob/master/examples/example1.cpp

https://arma.sourceforge.net/docs.html

https://anderkve.github.io/FYS3150/book/introduction_to_cpp/intro_to_armadillo.html

```cpp
//
// Efficient statistical computing with C++ Armadillo
//
// Maciek Bak
// Swiss Institute of Bioinformatics
// 15.07.2019
//

#include <iostream>
#include <armadillo>

//=============================================================================

void armadillo_toy_examples(){
  
  // Generate a vector from the standard normal distribution
  arma::vec v_normal = arma::randn(5);
  v_normal.print("v_normal:");

  // Create a 4x4 random matrix and print it on the screen
  arma::Mat<double> A = arma::randu(4,4);
  std::cout << "A:\n" << A << "\n";

  // Create a new diagonal matrix using the main diagonal of A:
  arma::Mat<double>B = arma::diagmat(A);
  std::cout << "B:\n" << B << "\n";

  // New matrix: directly specify the matrix size (elements are uninitialised)
  arma::mat C(2,3);  // typedef mat  =  Mat<double>
  std::cout << "C.n_rows: " << C.n_rows << std::endl;
  std::cout << "C.n_cols: " << C.n_cols << std::endl;

  // Directly access an element (indexing starts at 0)
  C(1,2) = 456.0;
  std::cout << "C[1][2]:\n" << C(1,2) << "\n";
  C = 5.5; // scalars are treated as a 1x1 matrix
  C.print("C:");

  // Inverse
  std::cout << "inv(C): " << std::endl << inv(C) << std::endl;
  
  // Rotate a point (0,1) by -Pi/2 ---> (1,0)
  arma::vec Position = {0,1};
  Position.print("Current coordinates of a point:");
  double Pi = 3.14159265359;
  double phi = -Pi/2;
  arma::mat RotationMatrix = {
    {+cos(phi), -sin(phi)},
    {+sin(phi), +cos(phi)}
  };
  Position = RotationMatrix * Position;
  Position.print("New coordinates of the point:");
}

//=============================================================================

void armadillo_read_write_objects(){

  // Create a 5x5 matrix with random values from uniform distribution on [0;1]
  // Save a double matrix to a csv format, then load it.
  arma::Mat<double> uniform_matrix = arma::randu(3,5);
  uniform_matrix.save("data/arma_uniform_matrix.csv", arma::csv_ascii);
  arma::Mat<double> load_matrix;
  load_matrix.load("data/arma_uniform_matrix.csv", arma::csv_ascii);

  // Armadillo can save directly to files or write to pre-opened streams.
  // In order to add column names to output tables we have to write the
  // header manually to a file stream and then save the matrix to the stream.
  std::ofstream file("data/arma_uniform_matrix_with_headers.csv");
  file << "A,B,C,D,E" << std::endl;
  uniform_matrix.save(file, arma::csv_ascii);
  file.close();
  //
  // As Armadillo objects are numerical structures the input shall not contain
  // row/column names. Armadillo should be used only for heavy computations.
}

//=============================================================================

int main(int argc, const char **argv){

  std::cout << "Armadillo version: " << arma::arma_version::as_string()
    << std::endl;

  // set RNG seed:
  arma::arma_rng::set_seed(0);
  //arma::arma_rng::set_seed_random();

  armadillo_toy_examples();

  armadillo_read_write_objects();

  return 0;
}

```

g++ example7.cpp -o example7 --std=c++17 -larmadillo  && ./example7

### Eigen

https://eigen.tuxfamily.org/index.php?title=Main_Page#Documentation

```cpp
//
// Efficient statistical computing with C++ Eigen
//
// Maciek Bak
// Swiss Institute of Bioinformatics
// 29.07.2019
//

#include <iostream>
#include <ctime>
#include "Eigen/Dense"

using namespace Eigen;

//=============================================================================

void eigen_toy_examples(){

  // Generate a vector from the standard normal distribution
  VectorXd v_normal = VectorXd::Random(5);
  std::cout << "v_normal:" << std::endl;
  std::cout << v_normal << std::endl;

  // Create a 4x4 random matrix and print it on the screen
  MatrixXd A = MatrixXd::Random(4, 4);
  std::cout << "A:" << std::endl;
  std::cout << A << std::endl;

  // Rotate a point (0,1) by -Pi/2 ---> (1,0)
  Vector2d Position(0, 1);
  std::cout << "Current coordinates of a point:" << std::endl;
  std::cout << Position << std::endl;
  double Pi = 3.14159265359;
  double phi = -Pi/2;
  // Rotation Matrix:
  Matrix2d RotationMatrix;
  RotationMatrix << +cos(phi), -sin(phi), +sin(phi), +cos(phi);
  Position = RotationMatrix * Position;
  std::cout << "New coordinates of the point:" << std::endl;
  std::cout << Position << std::endl;

}

//=============================================================================

int main(int argc, const char **argv){

  std::cout 
    << "Eigen version: "
    << EIGEN_WORLD_VERSION
    << "."
    << EIGEN_MAJOR_VERSION
    << "."
    << EIGEN_MINOR_VERSION
    << std::endl;

  // set RNG seed:
  std::srand(0);

  eigen_toy_examples();

  return 0;
}

```

```
g++ example8.cpp -o example8 --std=c++17  && ./example8
```

---